# Load Files

In [134]:
! pip install requests
! pip install requests-oauthlib
! pip install -q kaggle
! pip install plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [135]:
! rm -r ~/.kaggle
! mkdir ~/.kaggle
! mv ./kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mv: cannot stat './kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [136]:
! kaggle datasets download -d gpreda/reddit-wallstreetsbets-posts

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 166, in authenticate
    self.config_file, self.config_dir))
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [137]:
! ls

FRB_H6.csv  reddit-wallstreetsbets-posts.zip  reddit_wsb.csv  sample_data


In [138]:
! unzip reddit-wallstreetsbets-posts.zip

Archive:  reddit-wallstreetsbets-posts.zip
replace reddit_wsb.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: reddit_wsb.csv          


In [139]:
! ls

FRB_H6.csv  reddit-wallstreetsbets-posts.zip  reddit_wsb.csv  sample_data


# Imports

In [140]:
import pandas as pd
import numpy as np
from datetime import datetime

# data preprocessing
from sklearn import preprocessing

# exploratory analysis
import matplotlib.pyplot as plt
import mlxtend
from mlxtend.plotting import scatterplotmatrix
import seaborn as sns
import plotly.graph_objects as go

# model fit
import statsmodels.api as sm
import tensorflow as tf

# ignore warnings (libraries are rapidly changing)
import warnings
warnings.filterwarnings('ignore')

# EDA

In [141]:
# read 
money_stock_measures = pd.read_csv('FRB_H6.csv')

In [142]:
# read reddit wall street bets data
wsb_df = pd.read_csv('reddit_wsb.csv')

In [143]:
money_stock_measures.shape

(765, 30)

In [144]:
wsb_df.shape

(53187, 8)

In [145]:
money_stock_measures.head()

,Series Description,M1; Not seasonally adjusted,M2; Not seasonally adjusted,Currency; Not seasonally adjusted,Demand deposits; Not seasonally adjusted,Other liquid deposits - Total; Not seasonally adjusted,Small-denomination time deposits - Total; Not seasonally adjusted,Retail money market funds; Not seasonally adjusted,IRA and Keogh accounts at depository institutions; Not seasonally adjusted. Last 4 obs are estimates.,IRA and Keogh accounts at money market funds; Not seasonally adjusted. Last 4 obs are estimates.,...,Monetary base; total; not seasonally adjusted,"Reserves of depository institutions, total; not seasonally adjusted",Total borrowings from the Federal Reserve; not seasonally adjusted,"Reserves of depository institutions, nonborrowed; not seasonally adjusted",Other checkable deposits - Total; Not seasonally adjusted; *Discontinued after Apr 2020,Savings deposits - Total; Not seasonally adjusted; *Discontinued after Apr 2020,Travelers checks; Not seasonally adjusted; *Discontinued after Dec 2018,Other checkable deposits - Total; Seasonally adjusted; *Discontinued after Apr 2020,Savings deposits - Total; Seasonally adjusted; *Discontinued after Apr 2020,Travelers Checks; Seasonally adjusted; *Discontinued after Dec 2018
0,Unit:,Currency,Currency,Currency,Currency,Currency,Currency,Currency,Currency,Currency,...,Currency,Currency,Currency,Currency,Currency,Currency,Currency,Currency,Currency,Currency
1,Multiplier:,1e+09,1e+09,1e+09,1e+09,1e+09,1e+09,1e+09,1e+09,1e+09,...,1e+09,1e+09,1000000,1e+09,1e+09,1e+09,1e+09,1e+09,1e+09,1e+09
2,Currency:,USD,USD,USD,USD,USD,USD,USD,USD,USD,...,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD
3,Unique Identifier:,H6/H6_M1/M1_N.M,H6/H6_M2/M2_N.M,H6/H6_M1/MCU_N.M,H6/H6_M1/MDD_N.M,H6/H6_M1/MDL_N.M,H6/H6_M2/MDTS_N.M,H6/H6_M2/MMFGB_N.M,H6/H6_MEMO/MIKD_N.M,H6/H6_MEMO/MIKM_N.M,...,H6/H6_MBASE/RESMO14A_N.M,H6/H6_MEMO/RESTR14A_N.M,H6/H6_MEMO/RESBR14A_N.M,H6/H6_MEMO/RESNB14A_N.M,H6/H6_M1/MDO_N.M,H6/H6_M2/MDU_N.M,H6/H6_M1/MTC_N.M,H6/H6_M1/MDO.M,H6/H6_M2/MDU.M,H6/H6_M1/MTC.M
4,Time Period,M1_N.M,M2_N.M,MCU_N.M,MDD_N.M,MDL_N.M,MDTS_N.M,MMFGB_N.M,MIKD_N.M,MIKM_N.M,...,RESMO14A_N.M,RESTR14A_N.M,RESBR14A_N.M,RESNB14A_N.M,MDO_N.M,MDU_N.M,MTC_N.M,MDO.M,MDU.M,MTC.M


In [146]:
money_stock_measures.drop(index=money_stock_measures.index[:5], 
        axis=0, 
        inplace=True)

In [147]:
money_stock_measures = money_stock_measures.rename(columns={'M1; Not seasonally adjusted': 'M1', 'M2; Not seasonally adjusted':'M2', 'Time Period':'time_period','Series Description':'Date'})

In [148]:
money_stock_measures

,Date,M1,M2,Currency; Not seasonally adjusted,Demand deposits; Not seasonally adjusted,Other liquid deposits - Total; Not seasonally adjusted,Small-denomination time deposits - Total; Not seasonally adjusted,Retail money market funds; Not seasonally adjusted,IRA and Keogh accounts at depository institutions; Not seasonally adjusted. Last 4 obs are estimates.,IRA and Keogh accounts at money market funds; Not seasonally adjusted. Last 4 obs are estimates.,...,Monetary base; total; not seasonally adjusted,"Reserves of depository institutions, total; not seasonally adjusted",Total borrowings from the Federal Reserve; not seasonally adjusted,"Reserves of depository institutions, nonborrowed; not seasonally adjusted",Other checkable deposits - Total; Not seasonally adjusted; *Discontinued after Apr 2020,Savings deposits - Total; Not seasonally adjusted; *Discontinued after Apr 2020,Travelers checks; Not seasonally adjusted; *Discontinued after Dec 2018,Other checkable deposits - Total; Seasonally adjusted; *Discontinued after Apr 2020,Savings deposits - Total; Seasonally adjusted; *Discontinued after Apr 2020,Travelers Checks; Seasonally adjusted; *Discontinued after Dec 2018
5,1959-01,142.2,289.8,28.4,113.4,NaN,11.7,NaN,0.0,0.0,...,50.5,18.9,551.8,18.3,0.0,136.0,0.3,0.0,136.0,0.3
6,1959-02,139.3,287.7,28.2,110.8,NaN,11.7,NaN,0.0,0.0,...,49.8,18.6,505.0,18.1,0.0,136.7,0.3,0.0,136.6,0.3
7,1959-03,138.4,287.9,28.3,109.8,NaN,11.8,NaN,0.0,0.0,...,49.7,18.4,599.5,17.8,0.0,137.7,0.3,0.0,137.6,0.3
8,1959-04,139.7,290.2,28.3,111.1,NaN,12.0,NaN,0.0,0.0,...,50.1,18.7,691.6,18.0,0.0,138.5,0.3,0.0,138.4,0.3
9,1959-05,138.7,290.2,28.5,109.8,NaN,12.2,NaN,0.0,0.0,...,50.1,18.6,741.5,17.8,0.0,139.3,0.3,0.0,139.5,0.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
760,2021-12,20584.1,21653.5,2129.8,4769.8,13684.5,86.9,982.5,601.7,358.1,...,6413.1,4187.9,38082.2,4149.9,NaN,NaN,NaN,NaN,NaN,NaN
761,2022-01,20546.1,21646.4,2134.3,4758.9,13652.9,80.1,1020.2,602.7,351.8,...,6103.9,3871.1,32055.1,3839.1,NaN,NaN,NaN,NaN,NaN,NaN
762,2022-02,20534.3,21632.5,2142.8,4680.9,13710.6,69.0,1029.2,602.7,340.0,...,6040.0,3804.5,28714.6,3775.8,NaN,NaN,NaN,NaN,NaN,NaN
763,2022-03,20811.3,21925.7,2166.0,4800.6,13844.7,64.3,1050.1,602.7,328.2,...,6134.5,3874.7,26205.8,3848.5,NaN,NaN,NaN,NaN,NaN,NaN


In [149]:
money_stock_measures['Date'] = pd.to_datetime(money_stock_measures['Date'], format='%Y-%m', errors='coerce')
money_stock_measures['M1'] = pd.to_numeric(money_stock_measures['M1'])
money_stock_measures['M2'] = pd.to_numeric(money_stock_measures['M2'])

In [150]:
money_stock_measures.shape

(760, 30)

In [151]:
gb_year_m1 = money_stock_measures.groupby(money_stock_measures['Date'].dt.year)['M1'].agg(['sum', 'mean', 'max'])
gb_year_m2 = money_stock_measures.groupby(money_stock_measures['Date'].dt.year)['M2'].agg(['sum', 'mean', 'max'])

In [152]:
gb_year_m1

,sum,mean,max
Date,,,
1959,1684.4,140.366667,143.6
1960,1683.6,140.300000,144.5
1961,1716.8,143.066667,149.2
1962,1757.7,146.475000,151.9
1963,1811.1,150.925000,157.5
...,...,...,...
2018,44167.0,3680.583333,3800.7
2019,46143.8,3845.316667,4043.3
2020,153447.8,12787.316667,17949.4


In [153]:
gb_year_m2

,sum,mean,max
Date,,,
1959,3518.9,293.241667,300.6
1960,3651.7,304.308333,315.3
1961,3897.8,324.816667,338.5
1962,4200.9,350.075000,365.8
1963,4554.9,379.575000,396.4
...,...,...,...
2018,169247.1,14103.925000,14448.8
2019,177812.4,14817.700000,15412.5
2020,211842.0,17653.500000,19247.9


In [154]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = money_stock_measures['Date'],
                          y = money_stock_measures['M1'], 
                          fill=None))
fig.update_layout(
    title="Money Stock Measures M1",
    xaxis_title="Year (1959 - 2022)",
    yaxis_title="M1 - US Dollars (Billions)",
)
fig.show()

In [155]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = money_stock_measures['Date'],
                          y = money_stock_measures['M2'], 
                          fill=None))
fig.update_layout(
    title="Money Stock Measures M2",
    xaxis_title="Year (1959 - 2022)",
    yaxis_title="M2 - US Dollars (Billions)",
)
fig.show()

In [209]:
# read https://apps.bea.gov/iTable/iTable.cfm?reqid=19&step=2#reqid=19&step=2&isuri=1&1921=survey
gdp = pd.read_csv('GDP.csv')

In [210]:
gdp.head()

,Table 1.1.5. Gross Domestic Product,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64
0,[Billions of dollars],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Bureau of Economic Analysis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Last Revised on: May 26, 2022 - Next Release D...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Line,NaN,1959.0,1960.0,1961.0,1962.0,1963.0,1964.0,1965.0,1966.0,...,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0
4,1,Gross domestic product,521.7,542.4,562.2,603.9,637.5,684.5,742.3,813.4,...,16254.0,16843.2,17550.7,18206.0,18695.1,19479.6,20527.2,21372.6,20893.7,22996.1


In [211]:
gdp.drop(index=gdp.index[:3], 
        axis=0, 
        inplace=True)

In [212]:
gdp = gdp.iloc[: , 2:]

gdp.head()

,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,...,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64
3,1959.0,1960.0,1961.0,1962.0,1963.0,1964.0,1965.0,1966.0,1967.0,1968.0,...,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0
4,521.7,542.4,562.2,603.9,637.5,684.5,742.3,813.4,860.0,940.7,...,16254.0,16843.2,17550.7,18206.0,18695.1,19479.6,20527.2,21372.6,20893.7,22996.1
5,317.1,331.2,341.5,362.6,382.0,410.6,443.0,479.9,506.7,556.9,...,11047.4,11363.5,11847.7,12263.5,12693.3,13239.1,13913.5,14428.7,14047.6,15741.6
6,172.6,177.0,178.8,189.0,198.2,212.3,229.7,249.6,259.0,284.6,...,3637.7,3730.0,3863.0,3923.0,3991.8,4158.6,4353.7,4478.9,4653.8,5481.4
7,44.9,45.6,44.2,49.5,54.2,59.6,66.4,71.7,74.0,84.8,...,1144.2,1189.4,1242.1,1307.6,1345.2,1396.6,1469.2,1513.3,1616.4,2026.2


In [213]:
gdp_cleaned = pd.DataFrame({'Year' : gdp.iloc[0], 
                            'GDP' : gdp.iloc[1]},
                           columns=['Year', 'GDP'])
gdp_cleaned = gdp_cleaned.reset_index()
gdp_cleaned['Year'] = pd.to_datetime(gdp_cleaned['Year'], format='%Y', errors='coerce')
gdp_cleaned['GDP'] = pd.to_numeric(gdp_cleaned['GDP'])

In [214]:
gdp_cleaned

,index,Year,GDP
0,Unnamed: 2,1959-01-01,521.7
1,Unnamed: 3,1960-01-01,542.4
2,Unnamed: 4,1961-01-01,562.2
3,Unnamed: 5,1962-01-01,603.9
4,Unnamed: 6,1963-01-01,637.5
...,...,...,...
58,Unnamed: 60,2017-01-01,19479.6
59,Unnamed: 61,2018-01-01,20527.2
60,Unnamed: 62,2019-01-01,21372.6
61,Unnamed: 63,2020-01-01,20893.7


In [243]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = gdp_cleaned['Year'],
                          y = gdp_cleaned['GDP'], 
                          fill=None))
fig.update_layout(
    title="Gross Domestic Product",
    xaxis_title="Year (1959 - 2021)",
    yaxis_title="US Dollars (Billions)",
)
fig.show()

In [289]:
# read 
income = pd.read_csv('National_Income_and_Product.csv')

In [290]:
income.head()

,Table 2.1. Personal Income and Its Disposition,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64
0,[Billions of dollars],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Bureau of Economic Analysis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Last Revised on: May 26, 2022 - Next Release D...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Line,NaN,1959,1960,1961,1962,1963,1964,1965,1966.0,...,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0
4,s,Personal income,402.9,422.1,440.6,468.8,492.8,528.2,570.7,620.3,...,14014.3,14193.6,14976.6,15685.2,16096.9,16850.2,17706.0,18424.4,19627.6,21092.8


In [291]:
income.drop(index=income.index[:3], 
        axis=0, 
        inplace=True)

In [292]:
income = income.iloc[: , 2:]

income.head()

,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,...,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64
3,1959,1960,1961,1962,1963,1964,1965,1966.0,1967.0,1968.0,...,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0
4,402.9,422.1,440.6,468.8,492.8,528.2,570.7,620.3,665.7,730.9,...,14014.3,14193.6,14976.6,15685.2,16096.9,16850.2,17706.0,18424.4,19627.6,21092.8
5,285.8,301.3,310.4,332.2,350.4,376,405.4,449.2,481.8,530.8,...,8567.4,8835.0,9250.2,9699.4,9966.1,10426.1,10959.5,11447.7,11572.2,12598.7
6,259.8,272.9,280.5,299.4,314.9,337.8,363.8,400.3,429.0,472.0,...,6928.1,7114.0,7476.3,7859.5,8091.2,8474.7,8900.5,9323.5,9444.1,10343.8
7,213.8,223.7,228,243,254.8,272.9,293.8,321.9,342.5,375.3,...,5729.8,5906.0,6239.4,6583.7,6783.2,7126.7,7499.1,7873.3,7949.6,8808.3


In [293]:
income_cleaned = pd.DataFrame({'Year' : income.iloc[0], 
                            'National_Income' : income.iloc[1]},
                           columns=['Year', 'National_Income'])

In [294]:
#income_cleaned['Year'] = pd.to_datetime(income_cleaned['Year'], format='%Y', errors='coerce')
income_cleaned['National_Income'] = pd.to_numeric(income_cleaned['National_Income'])

In [295]:
income_cleaned

,Year,National_Income
Unnamed: 2,1959,402.9
Unnamed: 3,1960,422.1
Unnamed: 4,1961,440.6
Unnamed: 5,1962,468.8
Unnamed: 6,1963,492.8
...,...,...
Unnamed: 60,2017.0,16850.2
Unnamed: 61,2018.0,17706.0
Unnamed: 62,2019.0,18424.4
Unnamed: 63,2020.0,19627.6


In [296]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = income_cleaned['Year'],
                          y = income_cleaned['National_Income'], 
                          fill=None))
fig.update_layout(
    title="Personal Income",
    xaxis_title="Year (1959 - 2021)",
    yaxis_title="US Dollars (Billions)",
)
fig.show()

In [297]:
# read 
consumption = pd.read_csv('Personal_Consumption_Expenditures.csv')

In [298]:
consumption.head()

,Table 2.5.5. Personal Consumption Expenditures by Function,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63
0,[Billions of dollars],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Bureau of Economic Analysis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Last Revised on: July 30, 2021",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Line,NaN,1959,1960,1961,1962,1963,1964,1965,1966,...,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0
4,1,Personal consumption expenditures,317.1,331.2,341.5,362.6,382,410.6,443,479.9,...,10698.9,11047.4,11363.5,11847.7,12263.5,12693.3,13239.1,13913.5,14428.7,14047.6


In [299]:
consumption.drop(index=consumption.index[:3], 
        axis=0, 
        inplace=True)

In [300]:
consumption = consumption.iloc[: , 2:]

consumption.head()

,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,...,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63
3,1959,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0
4,317.1,331.2,341.5,362.6,382,410.6,443,479.9,506.7,556.9,...,10698.9,11047.4,11363.5,11847.7,12263.5,12693.3,13239.1,13913.5,14428.7,14047.6
5,312.3,326.1,336.2,357.1,376.3,404.2,436,472.5,498.6,548.1,...,10387.0,10705.8,11010.9,11482.1,11891.9,12291.8,12821.2,13468.9,13988.8,13526.7
6,61.6,62.6,63.7,64.7,65.9,69.5,74.4,80.6,82.6,88.8,...,819.5,846.2,864.0,896.9,921.0,940.6,973.1,1000.3,1030.9,1146.7
7,54,54.9,56.1,56.7,57.7,61,65.7,71.3,73,78.2,...,709.1,731.6,748.3,776.2,794.9,809.7,837.7,859.1,884.3,979.3


In [301]:
consumption_cleaned = pd.DataFrame({'Year' : consumption.iloc[0], 
                            'Consumption' : consumption.iloc[1]},
                           columns=['Year', 'Consumption'])

In [302]:
#consumption_cleaned['Year'] = pd.to_datetime(consumption_cleaned['Year'], format='%Y', errors='coerce')
consumption_cleaned['Consumption'] = pd.to_numeric(consumption_cleaned['Consumption'])

In [303]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = consumption_cleaned['Year'],
                          y = consumption_cleaned['Consumption'], 
                          fill=None))
fig.update_layout(
    title="Personal Consumption Expenditures",
    xaxis_title="Year (1959 - 2020)",
    yaxis_title="US Dollars (Billions)",
)
fig.show()